In [1]:
import pandas as pd
import mlflow.pytorch
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from graph_reinforcement_learning_using_blockchain_data import config
from graph_reinforcement_learning_using_blockchain_data.modeling import gnn

config.load_dotenv()

2025-05-07 17:42:21.498 | INFO     | graph_reinforcement_learning_using_blockchain_data.config:<module>:12 - PROJ_ROOT path is: /Users/liamtessendorf/Programming/Uni/2_Master/4_FS25_Programming/graph-reinforcement-learning-using-blockchain-data


True

In [2]:
dataset = torch.load(config.FLASHBOTS_Q2_DATA_DIR / "state_graphs_train_test_split.pt", weights_only=False)

In [3]:
max_feats_len = 0
for vals in dataset.values():
    for graphs in vals:
        if graphs.x.shape[1] > max_feats_len:
            max_feats_len = graphs.x.shape[1]

In [4]:
def pad_features(x, max_length):
    current_dim = x.size(1)
    if current_dim < max_length:
        pad_amount = max_length - current_dim
        x = F.pad(x, (0, pad_amount), "constant", 0)
    return x

In [5]:
for vals in dataset.values():
    for graphs in vals:
        graphs.x = pad_features(graphs.x, max_feats_len)

In [6]:
dataset["train_graphs"][5].x

tensor([[ 4.0000e+00,  4.0703e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  2.9232e-02,  7.8095e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0

In [7]:
train_dataset = dataset["train_graphs"]
test_dataset = dataset["test_graphs"]

In [8]:
train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True, exclude_keys=["account_mapping"], drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=256, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)
data_loader = DataLoader(
    dataset, batch_size=256, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)

In [9]:
num_node_features = max_feats_len
dim_global_features = 0
hidden_channels = 256  # adjust as needed
num_classes = 2  # binary classification

In [10]:
model_GNNSAGE = gnn.GraphSAGE(
    num_node_features + dim_global_features, hidden_channels, num_classes
)
print(model_GNNSAGE)

GraphSAGE(
  (conv1): SAGEConv(49, 256, aggr=mean)
  (conv2): SAGEConv(256, 256, aggr=mean)
  (conv3): SAGEConv(256, 256, aggr=mean)
  (conv4): SAGEConv(256, 256, aggr=mean)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [11]:
optimizer = torch.optim.Adam(model_GNNSAGE.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("mps")
model_GNNSAGE.to(device)

GraphSAGE(
  (conv1): SAGEConv(49, 256, aggr=mean)
  (conv2): SAGEConv(256, 256, aggr=mean)
  (conv3): SAGEConv(256, 256, aggr=mean)
  (conv4): SAGEConv(256, 256, aggr=mean)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [13]:
gnn.run_experiment(
    "GraphSAGE State Embeddings",
    20,
    model_GNNSAGE,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
)

2025/05/07 17:44:13 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 starts


  5%|▌         | 1/20 [00:58<18:35, 58.69s/it]

Epoch: 001, Train Loss: 0.0470, Test Acc: 0.9924
Epoch 2 starts


 10%|█         | 2/20 [02:15<20:52, 69.58s/it]

...

100%|██████████| 20/20 [1:19:04<00:00, 237.21s/it]

Epoch: 020, Train Loss: 0.0136, Test Acc: 0.9951



2025/05/07 19:03:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run efficient-crab-775 at: http://127.0.0.1:8080/#/experiments/748752183556303764/runs/d5126473f0714e61851be1b09c0d1d40
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/748752183556303764


(GraphSAGE(
   (conv1): SAGEConv(49, 256, aggr=mean)
   (conv2): SAGEConv(256, 256, aggr=mean)
   (conv3): SAGEConv(256, 256, aggr=mean)
   (conv4): SAGEConv(256, 256, aggr=mean)
   (fc1): Linear(in_features=256, out_features=128, bias=True)
   (fc2): Linear(in_features=128, out_features=2, bias=True)
 ),
 {})

In [23]:
model_uri = "mlflow-artifacts:/748752183556303764/465d6f94d00b4fe2bef4d8885ed7be39/artifacts/model"
model_GNNSAGE = mlflow.pytorch.load_model(model_uri)

In [22]:
_, _, embeddings = gnn.test(model_GNNSAGE, data_loader, criterion, device, return_embeddings=True)

In [30]:
emb = {trx_id: emb.cpu().detach().numpy().tolist() for trx_id, emb in embeddings.items()}

In [31]:
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

In [32]:
df_embeddings.to_csv(config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings.csv", index=False)